<a href="https://colab.research.google.com/github/lyapunovExponentGood/TRIPODS2022/blob/main/DiscreteDerivativeWithCustomizeableTimeDelays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following code contains a method, DerivativeCalc, for calculating the nth discrete derivative of a time series, where the time delay $\tau$ (the difference in indicies between terms being differenced) can be specified.  

These methods were written with improving time complexity in mind- anywhere where I could figure out how to replace a for loop with a numpy operation I did so. Because numpy is built on complex, finely optimized linear algebra libraries, this substitution should reduce time complexity. 

In [106]:
import math as mth
import numpy as np

In [107]:
#DerivativeCalc takes three parameters as inputs:
#array is the time series being differenced
#n is the number of derivatives, or differences that will be taken
#and tau being the time delay, or the difference between the indicies of terms being differenced

#it outputs an array containing the nth discrete derivative of the array with the provided time delay 
def DerivativeCalc(array,n,tau): 
  d = n + 1
  [embedding, d, tau] = TimeEmbedding(array,d,tau)

  binomialCoefs = np.zeros((d,1))
  coefVector = np.zeros((d,1))

  binomialCoefs[0:d,0] = BinomialCoefs(n)

  index = np.arange(d)
  sign = np.power(-1, index)
  coefVector[0:d,0] = np.multiply(binomialCoefs[0:d,0], sign) #the coefficents of d repeated differences of a sequence are provably  +/- binomial coefficents 

  derivativeList = np.dot(embedding, coefVector)

  return(derivativeList)


In [108]:
#TimeEmbedding takes a time series array, positive integer d, and positive integer tau
#and returns a nondisjoint delay time embedding of a time series with embedding dimension d and time delay tau

#Specifically, given a time series array [x_1, ... x_n] 
#it returns a n-tau*(d-1) x d matrix made of the d dimensional row vectors r_i = [x_i, x_{i+tau}, ..., x_{i+(d-1)*tau}] 
#the vector of d time series elements, in order, that are spaced tau time steps apart

def TimeEmbedding(array, d, tau): 
  n = len(array)
  m = n - tau*(d-1)
  #number of embeddings is the sequence length minus dimensionality-1 (d, points per embedding)
  #times the number of possible starting points for a unique embedding (tau, the time delay)

  [Rows, Cols] = np.indices((m,d))

  Index = Rows + tau*Cols

  Embedding = array[Index]
  embedding = Embedding[0:m, 0:d, 0]

  return [embedding, d, tau]

In [109]:
#BinomialCoefs takes a given positive integer n returns a vector of the binomial coefficents of n
def BinomialCoefs(n): 
  nArray = np.zeros(n+1) + n
  kArray = np.arange(n+1)

  nFactorial = Factorial(nArray)
  kFactorial = Factorial(kArray)
  nMinusKFactorial = np.flip(kFactorial)
  
  binomialCoefs = np.divide(nFactorial, np.multiply(nMinusKFactorial, kFactorial))

  return binomialCoefs

In [110]:
#Factorial takes an array of positive integers as an input and returns elementwise factorial of the elements in that array
def Factorial(kArray): 
  kFactorial = kArray
  index = np.where(kFactorial<1) 
  kFactorial[index] = 1 
  return(FactorialRecurse(kArray, kFactorial))

In [111]:
def FactorialRecurse(kArray, kFactorial):
  
  if(np.all(kArray <= 1)):
    return(kFactorial)
  else:
    kMinusOne = kArray - 1

    index = np.where(kMinusOne<1) 
    kMinusOne[index] = 1 #conserve all values that have already been factorialed 

    kFactorial = np.multiply(kFactorial, kMinusOne)
    
    #coefArray =
    return FactorialRecurse(kMinusOne, kFactorial)

**TESTING**: 

In [112]:
#simple test functions one can play with at their leisure

n=10

linear = np.zeros((n,1))
for i in range(0,n):
  linear[i] = i

quadratic = np.zeros((n,1))
for i in range(0,n):
  quadratic[i] = i**2

cubic = np.zeros((n,1))
for i in range(0,n):
  cubic[i] = i**3

#a(n) = cos(pi/2 *n)
cosine = np.zeros((1,20))
for i in range (0,20,2):
  cosine[0,i] = (-1)**(i/2) # discrete cos(n*pi/2)

cosine = np.transpose(cosine)

#a(n) = sin(pi/2 + n*pi)
sine = np.zeros((1,20))
for i in range (0,20):
  sine[0,i] = (-1)**(i) 

sine = np.transpose(sine)

In [114]:
n=1
tau=1
d1 = DerivativeCalc(quadratic, n, tau)
print(d1)

[[ -1.]
 [ -3.]
 [ -5.]
 [ -7.]
 [ -9.]
 [-11.]
 [-13.]
 [-15.]
 [-17.]]


**TESTING WITH SINUSOIDS:**

In [115]:
#odd time delay test:
tau = 3
[embeddingOdd, d, tau] = TimeEmbedding(sine,2,tau)
print(embeddingOdd)

[[ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [ 1. -1.]]


In [117]:
#even time delay test:
tau = 4
[embeddingEven, d, tau] = TimeEmbedding(sine,2,tau)
print(embeddingEven)

[[ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [ 1.  1.]
 [-1. -1.]]


In [120]:
dOdd = DerivativeCalc(sine,1,3)
dEven = DerivativeCalc(sine,1,4)
print(dOdd)
print(dEven)

[[ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
#makes sense: the nth derivative of sin for odd time delay tau is +\- 2^n 
#since each derivative essentially amounts to the binomial coefficents being added to each other n times, multiplied by 1 or negative 1

In [ ]:
#however, the nth derivatives for even time delay are zero as the time delay is equal to the period of the discrete sinuisoid 